# A/B Testing - Electronic House

## Descrição

### Contexto

A Electronic House é um comercio online ( e-commerce ) de produtos de informática para casas e escritórios. Os clientes podem comprar mouses, monitores, teclados, computadores, laptops, cabos HDMI, fones de ouvido, cameras webcam, entre outros, através de um site online e recebem os produtos no conforto de suas casas. O time de UX designers vem trabalhando em uma nova página de vendas, com o objetivo de aumentar a taxa de conversão de um produto da loja, um teclado bluetooth. O product manager (gerente de produto) disse que a taxa de conversão da página atual é de 13% em média, no último ano. O objetivo do product manager é aumentar a taxa de conversão em 2%, ou seja, a nova página de vendas, desenvolvida pelo time de UX, seria um sucesso se a sua taxa de conversão fosse de 15%. O teclado bluetooth possui um preço de venda de R$ 4.500,00 à vista ou parcelado em 12 % sem juros no cartão de crédito. Antes de trocar a página de vendas antiga pela nova, o product manager gostaria de testar a efetividade da nova página em um grupo menor de clientes, a fim de correr menos riscos de queda da conversão, caso a página nova mostre uma conversão pior do que a página atual.

### Desafio

Você foi contratado como um freelancer pela Electronic House para ajudar o time de
Designers da nova página, a validar a sua efetividade de uma maneira mais segura,
com mais confiança e rigidez na análise.
Os entregáveis do seu trabalho são os seguintes:
1. A conversão da nova página é realmente melhor do a conversão da página
atual?
2. Qual o potencial de número de vendas que a nova página pode trazer?
3. Qual o faturamento total na venda do teclado bluetooth através da nova página?

### Planejamento de Solução

**Passo 1: Escolha do método**

**1.1. Teste de hipóteses estatística.**

 Método de inferência estatística usado para decidir se os dados disponíveis são suficientes para suportar um hipóteses particular.

**1.2. Teste A/B**

Metodologia de pesquisa de experiência do usuário que aplica testes de hipóteses estatísticas para comparar duas ou mais versões de uma única variável, a fim de determinar qual das duas variantes é mais efetiva

**Passo 2: Design do Experimento**


**2.2. Escolha da variável.**
Definição da métrica de avaliação ou variável dependente.

**2.3. Separação dos grupos**
* Separação do grupo de controle.
* Separação do grupo de tratamento.
* Definição da tamanho da amostra de cada grupo.

**Passo 03: Coletando e preparando os dados**

**3.1. Coleta dos dados:**
* Definição da estrutura de coleta e armazenamento dos dados.
* Criação da Flag do A/B.
* Escolha das ferramentas de Teste A/B.

**3.2. Preparação dos dados:**
* Definição da estrutura de coleta e armazenamento dos dados.
* Limpeza e verificação dos dados.

**3.3. Conversões dos grupos:**
* Cálculo da conversão do grupo controle.
* Cálculo da conversão do grupo de tratamento.

**Passo 04: Testando as Hipóteses**

**4.1. Definição do método de inferência estatística**
* t-Test
* ANOVA
* Chi-Squared
* Cálculo do p-valor

**Passo 05: Tirando as conclusões**

* Interpretação do p-valor.
* Validação das hipóteses iniciais.
    * Conclusão.
    * Cálculo do número de vendas potencial.
    * Cálculo do faturamento pote

## 0. Imports

In [1]:
import pandas as pd

### 0.1. Helper Functions

In [2]:
pd.set_option('max_rows', 100)

### 0.2. Load Dataset

In [3]:
df_raw = pd.read_csv('../data/ab_testing.csv')

## 1. Data Description

In [4]:
df1 = df_raw.copy()

### 1.1. Data Dimensions

In [5]:
print('Nº of rows: {}'.format(df1.shape[0]))
print('Nº of cols: {}'.format(df1.shape[1]))

Nº of rows: 45883
Nº of cols: 8


### 1.2. Data Types

In [6]:
df1.dtypes

uid           int64
country      object
gender       object
spent         int64
purchases     int64
date         object
group        object
device       object
dtype: object

### 1.3. Change Dtypes

In [7]:
df1['date'] = pd.to_datetime(df1['date'])

### 1.4. Check NAs

In [8]:
df1.isna().sum()

uid          0
country      0
gender       0
spent        0
purchases    0
date         0
group        0
device       0
dtype: int64

### 1.5. Descriptive Statistics

In [9]:
num_attributes = df1[['spent', 'purchases']]
cat_attributes = df1.select_dtypes('object')

#### 1.5.1. Numerical Attributes 

In [10]:
num_attributes.describe()

,spent,purchases
count,45883.000000,45883.000000
mean,1883.453349,4.544385
std,1363.921546,3.102454
min,99.000000,1.000000
25%,897.000000,2.000000
50%,1596.000000,4.000000
75%,2593.000000,6.000000
max,12170.000000,30.000000


#### 1.5.2. Categorical Attributes

In [11]:
cat_attributes.describe()

,country,gender,group,device
count,45883,45883,45883,45883
unique,10,2,2,2
top,USA,M,GRP A,A
freq,13998,23116,23009,22982


##### 1.5.2.1. Group

In [12]:
df1['group'].value_counts(normalize=True)

GRP A    0.501471
GRP B    0.498529
Name: group, dtype: float64

##### 1.5.2.2. Gender

In [13]:
df1['gender'].value_counts(normalize=True)

M    0.503803
F    0.496197
Name: gender, dtype: float64

##### 1.5.2.3. Country

In [14]:
df1['country'].value_counts(normalize=True)

USA    0.305080
BRA    0.196347
MEX    0.120480
DEU    0.080553
TUR    0.077087
FRA    0.061940
GBR    0.061221
ESP    0.042151
CAN    0.032975
AUS    0.022165
Name: country, dtype: float64

# 2.0. Design do experimento

In [28]:
import math
from statsmodels.stats import api as sms
import pingouin as pg

## 2.1. Formulação de Hipóteses

## 2.2. Parâmetros de Experimentos

In [36]:
x = df1[df1['group'] =='GRP A']
y = df1[df1['group'] =='GRP B']

In [42]:
#nivel de confianca
confidence_level = 0.95

#nivel de significancia
significance_level = 0.05

p1 = 0.13
p2 = 0.15

# tamanho do efeito
effect_size = sms.proportion_effectsize(p1,p2)

#poder estatistico
power = 0.8

In [46]:
# sample size

sample_n = sms.NormalIndPower().solve_power(
                                 effect_size,
                                 power= power,
                                 alpha=significance_level
)
sample_n  = math.ceil(sample_n) 
sample_n

4720